# DEEPX Tutorial 05 - DX Compile workflow


In Tutorial 5, you will practice compiling classification, object detection, and segmentation models using the DX-Compiler. We will also review the guide for troubleshooting problems during compilation.

For more details, refer to the DX-COM user guide 👉 [Download](https://developer.deepx.ai/?files=MjUxNQ==)

>This tutorial is based on dx-all-suite v2.0.0, released in September 2025.

## Compiling Image Classification Model (MobileNetV2)
1. Export PyTorch → ONNX
2. JSON configuration for Input/Pre-processing/Calibration
3. Compile with DX-Compiler and verify .dxnn

### 1. Export PyTorch → ONNX

In [ ]:
# Install pytorch & onnx
!pip install --quiet torch torchvision onnx

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

Export Pytorch based MobileNetV2 model to ONNX

In [ ]:
import torch, torchvision

# Load MobileNetV2 model
model = torchvision.models.mobilenet_v2(weights=torchvision.models.MobileNet_V2_Weights.DEFAULT)
model.eval()

# Batch size must be 1
dummy_input = torch.randn(1, 3, 224, 224)

onnx_path = "MobilenetV2.onnx"

torch.onnx.export(
    model,                      # PyTorch model object to export
    dummy_input,                # Dummy input used for tracing (tuple is possible)
    onnx_path,                  # Output ONNX file path
    export_params=True,         # If True, saves model parameter (weight) into the ONNX file
    input_names=["input_test"], # Name of the ONNX model input tensor
    output_names=["output"],    # Name of the ONNX model output tensor
    opset_version=13            # ONNX opset version (recommended: 11 ~ 21)
)
print("✅ Save ONNX:", onnx_path)

### 2. JSON configuration for Input/Pre-processing/Calibration

Generate a configuration file for Input/Pre-processing/Calibration of MobilenetV2.

This JSON configuration file incudes:
 - Input specifications
 - Calibration methods
 - Data preprocessing settings
 - Optional parameters for advanced compilation schemes

Model **Input Restrictions**:
 - The batch size must be fixed to 1
 - Only a single input is supported (Multi-input will be supported in 2026)
 - Input name must exactly match ONNX model definition

#### 2.1. Incorrect input name case

This MobilenetV2 model's input name is `input_test` as shown below:

![](assets/mobilenetv2-input-name.png)

Input name (`input_test` in this example) must exactly match the input name of JSON configuration.

However, you can see there is wrong input name `incorrect_input_name` in the following JSON configuration for inputs.

In [ ]:
%%writefile MobilenetV2.json
{
  "inputs": {"incorrect_input_name": [1,3,224,224]},
  "calibration_num": 10,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 224, "height": 224}},
      {"div": {"x": 255.0}},
      {"normalize": {"mean": [0.485,0.456,0.406], "std": [0.229,0.224,0.225]}}
    ]
  }
}

Let's run DX-COM with this wrong JSON configuration. You will meet ERROR:
> ConfigInputError: The input name in config incorrect_input_name is not same as model input input_test

In [ ]:
!./dx_com/dx_com -m MobilenetV2.onnx -c MobilenetV2.json -o ./

#### 2.2. Incorrect input shape
In the following JSON file, the input name has the correct one - `input_test`.

However, even if your input shape is 1x3x224x224 (BxCxHxW), calibration image shape has 224x224x3 (HxWxC). You must change the shape of calibraiton image to match with 1x3x224x224 (BxCxHxW) by using `transpose` and `expandDim`.

In [ ]:
%%writefile MobilenetV2.json
{
  "inputs": {"input_test": [1,3,224,224]},
  "calibration_num": 10,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 224, "height": 224}},
      {"div": {"x": 255.0}},
      {"normalize": {"mean": [0.485,0.456,0.406], "std": [0.229,0.224,0.225]}}
    ]
  }
}

Let's run DX-COM with this wrong JSON configuration. You will meet ERROR:

> ConfigInputError: Config shape [1, 3, 224, 224] does not match preprocessed data shape [1, 224, 224, 3]

In [ ]:
!./dx_com/dx_com -m MobilenetV2.onnx -c MobilenetV2.json -o ./

#### 2.3. Add `transpose` & `expanDim` to JSON

In [ ]:
%%writefile MobilenetV2.json
{
  "inputs": {"input_test": [1,3,224,224]},
  "calibration_num": 10,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 224, "height": 224}},
      {"div": {"x": 255.0}},
      {"normalize": {"mean": [0.485,0.456,0.406], "std": [0.229,0.224,0.225]}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  }
}

### 3. Compile with DX-Compiler and verify .dxnn
Compile with dx_com to generate .dxnn

In [ ]:
!./dx_com/dx_com -m MobilenetV2.onnx -c MobilenetV2.json -o ./

In [ ]:
!run_model -m MobilenetV2.dxnn

## Compiling Object Detection Model (YOLOv9s)
1. Export PyTorch → ONNX
2. JSON configuration for Input/Pre-processing/Calibration
3. Compile with DX-Compiler and verify .dxnn

### 1. Export PyTorch → ONNX

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

#### 1.1. Download `yolov9-s.pt` pytorch model to your local system:

In [ ]:
#!cd yolov9 && pip install -r requirements.txt
!pip install torch==2.5.1 torchvision==0.20.1
!pip install onnx onnxsim

In [ ]:
!wget --no-check-certificate https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-s.pt

#### 1.2. Export the downloaded torch model to ONNX:

Download yolov9 git repo to use `export.py` for yolov9.

In [ ]:
!git clone https://github.com/WongKinYiu/yolov9.git

Export Pytorch based `yolov9-s` model to ONNX

In [ ]:
!cd yolov9 && python3 export.py --weights ../yolov9-s.pt \
                                --img-size 640 640 \
                                --opset 12 \
                                --simplify \
                                --batch-size 1 \
                                --include onnx

Check if `yolov9-s.onnx` file is generated.

In [ ]:
!ls | grep yolov9-s.onnx

### 2. JSON configuration for Input/Pre-processing/Calibration

In [ ]:
%%writefile yolov9-s.json
{
  "inputs": {"images": [1,3,640,640]},
  "calibration_num": 100,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpeg","jpg","png","JPEG"],
    "preprocessings": [
      {"resize": {"mode": "pad", "size": 640, "pad_location": "edge", "pad_value": [114,114,114]}},
      {"div": {"x": 255}},
      {"convertColor": {"form": "BGR2RGB"}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  }
}

### 3. Compile with DX-Compiler and verify .dxnn

In [ ]:
!./dx_com/dx_com -m yolov9-s.onnx -c yolov9-s.json -o ./

Check if **yolov9-s.dxnn** file is generated as expected:

In [ ]:
!ls | grep yolov9-s.dxnn

In [ ]:
!run_model -m yolov9-s.dxnn

## Compiling ViT Model

https://github.com/mlfoundations/open_clip


In [4]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

/home/max/Works/dx-tutorials/dx-all-suite/workspace/release/dx_com/dx_com_M1_v2.0.0


In [ ]:
!pip install git+https://github.com/openai/clip.git

In [ ]:
!pip install onnx onnxruntime

In [5]:
import clip
import torch

# 사용 가능한 모델 이름 확인
available_models = clip.available_models()
print(available_models)
# 출력 예: ['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 
#          'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']

# 장치 설정 (GPU 사용 가능 시 "cuda", 아닐 경우 "cpu")
device = "cuda" if torch.cuda.is_available() else "cpu"

# 특정 모델 로드 (예: "ViT-B/32")
# 이 시점에 모델이 로컬에 없으면 자동으로 다운로드됩니다.
model, preprocess = clip.load("ViT-B/32", device=device)

print("모델이 성공적으로 로드되었습니다.")

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']
모델이 성공적으로 로드되었습니다.


In [6]:
import torch.onnx
import os
import onnxruntime
import numpy as np
from PIL import Image

model.eval() # 모델을 평가 모드로 설정 (필수)
print("CLIP model loaded successfully.")

# --- 2. ONNX 내보내기 설정 ---
ONNX_OPSET_VERSION = 20
IMAGE_ONNX_PATH = "clip_image_encoder.onnx"
TEXT_ONNX_PATH = "clip_text_encoder.onnx"

# 모델의 입력 해상도 및 컨텍스트 길이 가져오기
image_resolution = model.visual.input_resolution
context_length = model.context_length
model_dtype = model.dtype # 모델의 데이터 타입 (cuda일 경우 float16, cpu일 경우 float32)

print(f"Model dtype: {model_dtype}")
print(f"Image resolution: {image_resolution}")
print(f"Text context length: {context_length}")

# --- 3. Image Encoder (Visual) ONNX 변환 ---
print(f"\nExporting Image Encoder to {IMAGE_ONNX_PATH}...")

# 1. 더미 이미지 입력 생성
#    배치 크기 1, 3개 채널, 모델 해상도
dummy_image_input = torch.randn(
    1, 3, image_resolution, image_resolution, 
    device=device, dtype=model_dtype
)

# 2. ONNX 변환 실행
torch.onnx.export(
    model.visual,                 # 내보낼 모델 (Image Encoder)
    dummy_image_input,            # 더미 입력
    IMAGE_ONNX_PATH,              # 저장 경로
    opset_version=ONNX_OPSET_VERSION,
    input_names=['image'],        # 입력 레이어 이름
    output_names=['image_features'] # 출력 레이어 이름
)
print("Image Encoder export complete.")


# --- 4. Text Encoder ONNX 변환 ---
print(f"\nExporting Text Encoder to {TEXT_ONNX_PATH}...")

# 1. Text Encoder 래퍼 클래스 정의
# 이유: model.encode_text는 단순 nn.Module이 아니라, 
#      여러 모듈(embedding, transformer, projection)을 순차적으로 호출하는 '메서드'입니다.
#      ONNX로 내보내려면 이 로직을 forward() 메서드로 감싸야 합니다.
class TextEncoderWrapper(torch.nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.transformer = clip_model.transformer
        self.token_embedding = clip_model.token_embedding
        self.positional_embedding = clip_model.positional_embedding
        self.ln_final = clip_model.ln_final
        self.text_projection = clip_model.text_projection
        self.dtype = clip_model.dtype

    def forward(self, text):
        # clip.model.CLIP.encode_text의 로직을 그대로 구현
        x = self.token_embedding(text).type(self.dtype)  # [batch_size, n_ctx, d_model]
        
        # positional_embedding은 [n_ctx, d_model]
        x = x + self.positional_embedding[:x.size(1)].type(self.dtype)
        
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = self.ln_final(x).type(self.dtype)

        # EOT 토큰의 피처를 가져옵니다.
        # clip.tokenize()는 EOT 토큰(49407)이 가장 크고 패딩은 0으로 채우므로 argmax로 찾습니다.
        batch_indices = torch.arange(x.size(0), device=text.device)
        eot_indices = text.argmax(dim=-1)
        x = x[batch_indices, eot_indices] 
        
        # 최종 프로젝션
        x = x @ self.text_projection
        return x

# 2. 래퍼 인스턴스 생성 및 더미 텍스트 입력 생성
text_encoder = TextEncoderWrapper(model).to(device).eval()
dummy_text_input = clip.tokenize(["a test sentence for onnx export"]).to(device) # (1, 77)

# 3. ONNX 변환 실행
torch.onnx.export(
    text_encoder,                 # 내보낼 모델 (Text Encoder 래퍼)
    dummy_text_input,             # 더미 입력
    TEXT_ONNX_PATH,               # 저장 경로
    opset_version=ONNX_OPSET_VERSION,
    input_names=['text'],         # 입력 레이어 이름 (토큰 ID)
    output_names=['text_features'] # 출력 레이어 이름
)
print("Text Encoder export complete.")


# --- 5. (선택) ONNX 모델 검증 ---
print("\nVerifying ONNX models...")

# CPU에서 numpy로 비교 (GPU/CPU, fp16/fp32 타입 불일치 문제 방지)
device_for_verify = "cpu"
model.to(device_for_verify)
text_encoder.to(device_for_verify)

# PyTorch 원본 모델 출력
with torch.no_grad():
    # 실제 테스트용 입력
    test_image_input_dummy = torch.randn(1, 3, image_resolution, image_resolution, device=device_for_verify)
    test_text_input = clip.tokenize(["a photo of a cat"]).to(device_for_verify) # (1, 77)

    # 이미지 모델은 fp32로 변환 후 실행 (ONNX 런타임이 보통 fp32로 실행됨)
    torch_image_out = model.visual(test_image_input_dummy.float()).cpu().numpy()
    
    # 텍스트 모델 실행
    torch_text_out = text_encoder(test_text_input).cpu().numpy()


# ONNX 런타임 세션 생성 및 출력
ort_image_session = onnxruntime.InferenceSession(IMAGE_ONNX_PATH, providers=['CPUExecutionProvider'])
ort_text_session = onnxruntime.InferenceSession(TEXT_ONNX_PATH, providers=['CPUExecutionProvider'])

# ONNX 모델 실행
# onnx 런타임은 numpy 배열을 입력으로 받습니다.
ort_image_input = {'image': test_image_input_dummy.float().cpu().numpy()}
ort_text_input = {'text': test_text_input.cpu().numpy()}

ort_image_out = ort_image_session.run(['image_features'], ort_image_input)[0]
ort_text_out = ort_text_session.run(['text_features'], ort_text_input)[0]

# 결과 비교
try:
    # atol (절대 오차) 값을 1e-4 또는 1e-5 정도로 설정
    np.testing.assert_allclose(torch_image_out, ort_image_out, rtol=1e-03, atol=1e-04)
    print("✅ Image Encoder ONNX verification SUCCESSFUL!")
except AssertionError as e:
    print(f"❌ Image Encoder ONNX verification FAILED: {e}")

try:
    np.testing.assert_allclose(torch_text_out, ort_text_out, rtol=1e-03, atol=1e-04)
    print("✅ Text Encoder ONNX verification SUCCESSFUL!")
except AssertionError as e:
    print(f"❌ Text Encoder ONNX verification FAILED: {e}")

print("\nAll tasks finished.")

CLIP model loaded successfully.
Model dtype: torch.float32
Image resolution: 224
Text context length: 77

Exporting Image Encoder to clip_image_encoder.onnx...
Image Encoder export complete.

Exporting Text Encoder to clip_text_encoder.onnx...
Text Encoder export complete.

Verifying ONNX models...
✅ Image Encoder ONNX verification SUCCESSFUL!
✅ Text Encoder ONNX verification SUCCESSFUL!

All tasks finished.


In [15]:
# --- 6. ONNX 모델을 이용한 실제 추론 (End-to-End) ---

# onnxruntime, numpy, PIL, clip, urllib 등 필요한 모듈 재확인
import onnxruntime
import numpy as np
from PIL import Image
import clip
import torch
import urllib.request
import os
import time

print("\n--- Starting Step 6: End-to-End ONNX Inference ---")

# --- 6-1. 설정 및 모델 로드 ---

# 원본 PyTorch 모델에서 전처리기(preprocess)와 logit_scale을 가져옵니다.
# (사용자님의 기존 코드에서 'model'과 'preprocess' 변수가 이미 로드되었다고 가정합니다)
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model, preprocess = clip.load("ViT-B/32", device=device) # 이 부분이 이미 실행됨

# ONNX 파일 경로
IMAGE_ONNX_PATH = "clip_image_encoder.onnx"
TEXT_ONNX_PATH = "clip_text_encoder.onnx"

# ONNX 런타임 세션 로드 (CPUExecutionProvider 사용)
# (검증 단계에서 이미 생성했지만, 추론 파이프라인을 위해 명시적으로 다시 로드)
providers = ['CPUExecutionProvider']
image_session = onnxruntime.InferenceSession(IMAGE_ONNX_PATH, providers=providers)
text_session = onnxruntime.InferenceSession(TEXT_ONNX_PATH, providers=providers)

# CLIP의 logit_scale 값을 원본 모델에서 가져옵니다.
# 이 값은 유사도(cosine similarity)를 최종 확률로 변환할 때 필요합니다.
# .item()을 사용해 Python 숫자로 변환합니다.
# 이미 device_for_verify = "cpu"로 모델을 옮겼으므로 .cpu()는 필요 없을 수 있습니다.
try:
    logit_scale = model.logit_scale.exp().item()
except AttributeError:
    # 혹시 model이 CPU에 없는 경우 대비
    logit_scale = model.to("cpu").logit_scale.exp().item()

print(f"Loaded ONNX sessions and logit_scale ({logit_scale:.4f})")


# --- 6-2. 헬퍼 함수 정의 (Numpy 기반) ---

def normalize(features):
    """Numpy 배열을 L2 정규화합니다."""
    norm = np.linalg.norm(features, axis=1, keepdims=True)
    return features / norm

def softmax(x):
    """Numpy 배열에 softmax를 적용합니다."""
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / e_x.sum(axis=-1, keepdims=True)


# --- 6-3. 테스트 데이터 준비 ---

# 1. 샘플 이미지 다운로드 및 전처리
image_url = "http://images.cocodataset.org/val2017/000000039769.jpg" # 고양이 사진
image_path = "cat_sample.jpg"

if not os.path.exists(image_path):
    print(f"Downloading sample image to {image_path}...")
    urllib.request.urlretrieve(image_url, image_path)

# PIL 이미지 열기
pil_image = Image.open(image_path)

# CLIP의 'preprocess' 함수로 이미지 전처리
# preprocess는 (224, 224) 크기의 PyTorch 텐서를 반환합니다.
image_tensor = preprocess(pil_image).unsqueeze(0) # (1, 3, 224, 224)

# ONNX 런타임은 numpy 배열을 입력으로 받습니다.
# 검증 단계(5)에서 .float()을 사용했으므로 여기서도 동일하게 fp32로 변환합니다.
# (원본 모델이 fp16이었더라도 ONNX가 fp32 입력을 처리할 수 있음)
image_input_np = image_tensor.float().cpu().numpy()
print(f"Image preprocessed. Shape: {image_input_np.shape}, Dtype: {image_input_np.dtype}")


# 2. 텍스트 레이블 준비 및 토큰화
text_labels = ["a photo of a dog", "a photo of a cat", "a photo of a bird", "a car"]
text_tokens = clip.tokenize(text_labels) # (4, 77)

# ONNX 런타임용 numpy 배열로 변환 (int64)
text_input_np = text_tokens.cpu().numpy()
print(f"Text tokenized. Shape: {text_input_np.shape}, Dtype: {text_input_np.dtype}")


# --- 6-4. ONNX 추론 실행 ---

# 1. 이미지 피처 추출
image_input_dict = {'image': image_input_np}
image_features_np = image_session.run(['image_features'], image_input_dict)[0]
print(f"Image features extracted. Shape: {image_features_np.shape}")

# 2. 텍스트 피처 추출
text_features_list = []
print(f"Text batch size is {text_input_np_int64.shape[0]}. Running inference in a loop...")

for i in range(text_input_np_int64.shape[0]):
    # [i:i+1] 슬라이싱을 사용해 (1, 77) 형태를 유지하며 하나씩 추출
    single_text_input = text_input_np[i:i+1]
    # 안전을 위해 입력 타입을 np.int32로 명시적으로 변환합니다.
    single_text_input_int32 = single_text_input.astype(np.int32)
    
    text_input_dict = {'text': single_text_input_int32}
    # 텍스트 피처를 하나씩 추론
    single_text_features = text_session.run(['text_features'], text_input_dict)[0]
    
    text_features_list.append(single_text_features)

# 추론된 피처들을 다시 하나의 numpy 배열로 합칩니다.
text_features_np = np.vstack(text_features_list)
print(f"Text features extracted (from loop). Shape: {text_features_np.shape}")


# --- 6-5. 유사도 계산 및 결과 확인 ---

# 1. 피처 정규화 (L2 Normalization)
image_features_norm = normalize(image_features_np)
text_features_norm = normalize(text_features_np)

# 2. 코사인 유사도 계산 (행렬 곱)
# (1, 512) @ (512, 4) -> (1, 4)
similarity_scores = image_features_norm @ text_features_norm.T
print(f"\nCosine Similarity Scores (raw):\n{similarity_scores}")

# 3. 최종 확률 계산 (logit_scale 적용 및 softmax)
logits = similarity_scores * logit_scale
probabilities = softmax(logits)

print(f"\n--- Final Probabilities ---")
for label, prob in zip(text_labels, probabilities[0]):
    print(f"{label:<20}: {prob * 100:.2f}%")

# 결과 확인
best_match_index = np.argmax(probabilities[0])
print(f"\n✅ Best Match: {text_labels[best_match_index]}")

print("\n--- ONNX End-to-End Inference Finished ---")


--- Starting Step 6: End-to-End ONNX Inference ---
Loaded ONNX sessions and logit_scale (100.0000)
Image preprocessed. Shape: (1, 3, 224, 224), Dtype: float32
Text tokenized. Shape: (4, 77), Dtype: int32
Image features extracted. Shape: (1, 512)
Text batch size is 4. Running inference in a loop...
Text features extracted (from loop). Shape: (4, 512)

Cosine Similarity Scores (raw):
[[0.19304886 0.24570042 0.17482775 0.18901584]]

--- Final Probabilities ---
a photo of a dog    : 0.51%
a photo of a cat    : 99.06%
a photo of a bird   : 0.08%
a car               : 0.34%

✅ Best Match: a photo of a cat

--- ONNX End-to-End Inference Finished ---


In [8]:
%%writefile clip_image.json
{
  "inputs":
  {
    "image": [1, 3, 224, 224]},
    "default_loader": 
    {
      "dataset_path": "/home/max/Works/dx-tutorials/dx-all-suite/dx-compiler/dx_com/calibration_dataset",
      "file_extensions": ["jpeg", "jpg", "png", "JPEG"],
      "preprocessings": [{"resize": {"width": 224, "height": 224}},
                         {"centercrop": {"width": 224, "height": 224}},
                         {"convertColor": {"form": "BGR2RGB"}},
                         {"div": {"x": 255.0}},
                         {"normalize":
                           {"mean": [0.48145466, 0.4578275, 0.40821073],
                            "std": [0.26862954, 0.26130258, 0.27577711]}},
                         {"transpose": {"axis": [2, 0, 1]}},
                         {"expandDim": {"axis": 0}}]},
  "calibration_num": 100,
  "calibration_method": "ema",
  "train_batchsize": 32,
  "num_samples": 100
}


Overwriting clip_image.json


In [10]:
!ls

calibration_dataset	 clip_image.json	 dx_com   Makefile  sample
clip_image_encoder.onnx  clip_text_encoder.onnx  LICENSE  output


In [11]:
!./dx_com/dx_com -m clip_image_encoder.onnx -c clip_image.json -o output

[INFO] - Logger initialized in 'release' mode.
[INFO] - Starting ONNX import for: clip_image_encoder.onnx                      
[INFO] - Loading and validating model from: clip_image_encoder.onnx             
[INFO] - Model loaded and validated successfully.                               
[INFO] - Converting model opset to target 21...                                 
[INFO] - Opset conversion successful.                                           
[INFO] - Updating model IR version from 9 to 10.                                
[INFO] - Successfully imported ONNX model!!!                                    
[INFO] - --- Running Surgeon Stage ---                                          
[INFO] - --- Running Canonicalizer Passes for NPU M1A ---                       
[INFO] - Running PassManager with 48 rules.                                     
[INFO] - PassManager finished.                                                  
[INFO] - Canonicalizer finished. Applied 48 pass(es).         

In [12]:
!tree output

output
└── clip_image_encoder.dxnn

1 directory, 1 file


In [22]:
!cd ../../../../dx-runtime/dx_rt/python_package && pip install .

Processing /home/max/Works/dx-tutorials/dx-all-suite/dx-runtime/dx_rt/python_package
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dx-engine: filename=dx_engine-1.1.2-cp312-cp312-linux_x86_64.whl size=189899 sha256=2989057066156991f3888076a43a6656a6d54d7ab48042dabab11d5ee50b6b96
  Stored in directory: /tmp/pip-ephem-wheel-cache-uixlx5bt/wheels/1c/bb/97/e08a0a8549cdadcbb71bf23d8fc888c1fc70f706f873c48d66
Successfully built dx-engine


In [39]:
from dx_engine import InferenceEngine
import cv2

IMAGE_PATH = "cat_sample.jpg"
IMAGE_RESOLUTION = 224 # "ViT-B/32" 모델의 해상도

# CLIP 모델 학습 시 사용된 표준 정규화 값 (필수)
CLIP_MEAN = np.array([0.48145466, 0.4578275, 0.40821073], dtype=np.float32)
CLIP_STD = np.array([0.26862954, 0.26130258, 0.27577711], dtype=np.float32)

# --- 2. 이미지 로드 (OpenCV) ---
# cv2.imread()는 (H, W, C) 순서이며, BGR 형식으로 로드됩니다.
image = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

if image is None:
    print(f"오류: '{IMAGE_PATH}' 파일을 찾을 수 없습니다.")
else:
    # --- 3. 전처리 (BGR -> RGB) ---
    # CLIP 모델은 RGB 입력을 기준으로 학습되었습니다.
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # --- 4. 전처리 (Resize & Center Crop) ---
    # clip.preprocess의 Resize(interpolation=BICUBIC) 및 CenterCrop 재현
    
    h, w, _ = image_rgb.shape
    
    # 짧은 쪽을 224로 맞추고 비율 유지
    if h < w:
        new_h = IMAGE_RESOLUTION
        new_w = int(w * (new_h / h)) # 비율에 맞게 긴 쪽 계산
    else:
        new_w = IMAGE_RESOLUTION
        new_h = int(h * (new_w / w))
        
    # 4-1. Resize (torchvision.transforms.Resize의 BICUBIC -> cv2.INTER_CUBIC)
    resized_image = cv2.resize(image_rgb, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
    
    # 4-2. Center Crop
    top = (new_h - IMAGE_RESOLUTION) // 2
    left = (new_w - IMAGE_RESOLUTION) // 2
    
    cropped_image = resized_image[top:top + IMAGE_RESOLUTION, left:left + IMAGE_RESOLUTION]

    # --- 5. 전처리 (정규화 및 타입 변환) ---
    # HWC, [0, 255] uint8 -> HWC, [0.0, 1.0] float32
    image_float = cropped_image.astype(np.float32) / 255.0
    
    # 5-1. 정규화 (Normalize)
    normalized_image = (image_float - CLIP_MEAN) / CLIP_STD

    # --- 6. 전처리 (HWC -> CHW) ---
    # (224, 224, 3) -> (3, 224, 224)
    #image_chw = np.transpose(normalized_image, (2, 0, 1))
    image_chw = np.transpose(image_float, (2, 0, 1))

    # --- 7. 전처리 (배치 차원 추가) ---
    # (3, 224, 224) -> (1, 3, 224, 224)
    # ONNX 런타임에 입력할 최종 배열
    image_batch = np.expand_dims(image_chw, axis=0).astype(np.float32)

    print(f"전처리 완료!")
    print(f"입력 파일: {IMAGE_PATH}")
    print(f"최종 배열 Shape: {image_batch.shape}")
    print(f"최종 배열 Dtype: {image_batch.dtype}")

dxnn_model_path = "output/clip_image_encoder.dxnn"

ie = InferenceEngine(dxnn_model_path)

#input = [image_input_np]
input = [image_batch]
outputs = ie.run(input)
#print(outputs)
print(f"Image features extracted. Shape: {outputs[0].shape}")

#image_features_norm = normalize(outputs[0])
#text_features_norm = normalize(text_features_np)
image_features_norm = normalize(outputs[0])
text_features_norm = normalize(text_features_np)

# 2. 코사인 유사도 계산 (행렬 곱)
# (1, 512) @ (512, 4) -> (1, 4)
similarity_scores = image_features_norm @ text_features_norm.T
print(f"\nCosine Similarity Scores (raw):\n{similarity_scores}")

# 3. 최종 확률 계산 (logit_scale 적용 및 softmax)
logits = similarity_scores * logit_scale
probabilities = softmax(logits)

print(f"\n--- Final Probabilities ---")
for label, prob in zip(text_labels, probabilities[0]):
    print(f"{label:<20}: {prob * 100:.2f}%")

# 결과 확인
best_match_index = np.argmax(probabilities[0])
print(f"\n✅ Best Match: {text_labels[best_match_index]}")

print("\n--- ONNX End-to-End Inference Finished ---")

전처리 완료!
입력 파일: cat_sample.jpg
최종 배열 Shape: (1, 3, 224, 224)
최종 배열 Dtype: float32
Image features extracted. Shape: (1, 512)

Cosine Similarity Scores (raw):
[[0.24744876 0.24209568 0.2440253  0.2488713 ]]

--- Final Probabilities ---
a photo of a dog    : 29.00%
a photo of a cat    : 16.98%
a photo of a bird   : 20.59%
a car               : 33.43%

✅ Best Match: a car

--- ONNX End-to-End Inference Finished ---


In [ ]:
import torch
import open_clip
from open_clip import tokenizer as clip_tokenizer
DEVICE = "cpu"  # export 용으론 CPU 권장
MODEL_NAME = "ViT-L-14-quickgelu"
PRETRAINED = "dfn2b"   # 필요 시 원하는 태그로 교체
# 1) 모델/전처리 로드 (가중치 자동 다운로드 & 캐시)
model, _, preprocess = open_clip.create_model_and_transforms(
    MODEL_NAME, pretrained=PRETRAINED, device=DEVICE
)
model.eval()
# 2) 이미지/텍스트 인코더 래퍼 정의 (프로젝션 포함, L2 정규화 비활성화)
class ImageEncoder(torch.nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.visual = clip_model.visual
        self.image_proj = getattr(clip_model, "image_projection", None)
        self.normalize = torch.nn.functional.normalize
    def forward(self, x):
        # x: [1,3,H,W] (preprocess로 정규화된 텐서)
        feats = self.visual(x)                      # pooled visual features
        if self.image_proj is not None:
            feats = feats @ self.image_proj         # projection to embed_dim
        # 배포 파이프라인에서 보통 후처리로 L2 정규화하므로 ONNX엔 비정규화 출력
        return feats
class TextEncoder(torch.nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.text = clip_model.transformer
        self.text_projection = getattr(clip_model, "text_projection", None)
        self.ln_final = clip_model.ln_final
        self.token_embedding = clip_model.token_embedding
        self.positional_embedding = clip_model.positional_embedding
        self.register_buffer("attn_mask", clip_model.attn_mask)
    def forward(self, tokens):
        # tokens: [1, 77] (int32/int64)
        x = self.token_embedding(tokens) + self.positional_embedding
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.text(x, attn_mask=self.attn_mask)
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = self.ln_final(x)
        # EOT 토큰의 임베딩 선택 (OpenAI CLIP 규약)
        eot_indices = tokens.argmax(dim=-1)
        x = x[torch.arange(x.shape[0]), eot_indices]  # [N, hidden]
        if self.text_projection is not None:
            x = x @ self.text_projection
        return x
img_encoder = ImageEncoder(model).to(DEVICE).eval()
txt_encoder = TextEncoder(model).to(DEVICE).eval()
# 3) 더미 입력 (배치=1 고정)
# 이미지 크기: 모델에 내장된 입력 해상도 사용
image_size = getattr(model.visual, "image_size", 224)
dummy_image = torch.randn(1, 3, image_size, image_size, device=DEVICE)
dummy_tokens = clip_tokenizer.tokenize(["dummy text"]).to(DEVICE)  # [1,77]
# 4) ONNX 내보내기
common = dict(
    opset_version=17,          # 17 이상 권장
    export_params=True,
    do_constant_folding=True,
    dynamic_axes=None          # 배치=1 고정
)
with torch.no_grad():
    torch.onnx.export(
        img_encoder, dummy_image,
        f"clip_{MODEL_NAME}_image_{PRETRAINED}.onnx",
        input_names=["pixel_values"],
        output_names=["image_embeds"],
        **common
    )
    torch.onnx.export(
        txt_encoder, dummy_tokens,
        f"clip_{MODEL_NAME}_text_{PRETRAINED}.onnx",
        input_names=["input_ids"],
        output_names=["text_embeds"],
        **common
    )
print("DONE :흰색_확인_표시:")

open_clip_pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [ ]:
import os
import sys
import torch
import numpy as np

import open_clip
from open_clip.zero_shot_classifier import build_zero_shot_classifier

def import_imagenet_metadata():
    try:
        from open_clip.zero_shot_metadata import (
            IMAGENET_CLASSNAMES,
            OPENAI_IMAGENET_TEMPLATES,
        )
        return IMAGENET_CLASSNAMES, OPENAI_IMAGENET_TEMPLATES
    except ImportError:
        print("\n[ERROR] Could not import IMAGENET_CLASSNAMES or OPENAI_IMAGENET_TEMPLATES.")
        print("This suggests a mismatch between your source code and the installed library in your conda environment.")
        sys.exit(1)

device = 'cpu'
model_name = 'ViT-L-14-quickgelu'
pre_trained = 'dfn2b'
output_path = os.path.join('./', f"{model_name}-{pre_trained}.npy")

# Import classnames and templates
classnames, templates = import_imagenet_metadata()

# Load model and tokenizer
model, _, _ = open_clip.create_model_and_transforms(
    model_name, pretrained=pre_trained, device=device
)
tokenizer = open_clip.get_tokenizer(model_name)
model.eval()

# Build zero-shot classifier weights
print(f"Building zero-shot classifier weights on '{device}'...")
with torch.no_grad():
    zeroshot_weights = build_zero_shot_classifier(
        model,
        tokenizer=tokenizer,
        classnames=classnames,
        templates=templates,
        device=device,
    )

print(f"Zero-shot weights shape: {zeroshot_weights.shape}, dtype: {zeroshot_weights.dtype}")

# Save as numpy
os.makedirs(output_dir, exist_ok=True)
np.save(output_path, zeroshot_weights.cpu().numpy())
print(f"Zero-shot weights saved to: {output_path}")
print("Done!")

Building zero-shot classifier weights on 'cpu'...
